In [ ]:
%load_ext autoreload
%autoreload 2

# Experiments with the JointTracker class

In [ ]:
from climbing_wire.homography.homography import compute_homography
from climbing_wire.homography.homography import perspective_transform
from climbing_wire.landmark.compute import PoseImg
from climbing_wire.landmark.compute import compute_landmarks
from climbing_wire.landmark.drawing import draw_landmarks
from climbing_wire.landmark.landmark_list import LandmarkListImg
from climbing_wire.utils.cv import cv_imshow
from climbing_wire.utils.data import get_package_fol
from climbing_wire.utils.mediapipe import JOINT_NAMES
from climbing_wire.utils.mediapipe import get_default_pose_connections
from climbing_wire.video.load import iterate_video_frames, pairwise_video_frames
from matplotlib import pyplot as plt
from pathlib import Path
from typing import List, Tuple, cast
import cv2 as cv
import math
import matplotlib.pyplot as plt
import mediapipe as mp
import mediapipe.python.solutions.drawing_styles as mp_drawing_styles
import mediapipe.python.solutions.drawing_utils as mp_drawing
import mediapipe.python.solutions.pose as mp_pose
import numpy as np
import pathlib
import sys


In [ ]:
# in_fol = Path("/mnt/c/Users/nobilip/Videos/dataset/rock")
# in_fn = "PXL_20220901_202119640.mp4"

in_fol = Path("~/data/rock").expanduser()
in_fn = "yoshiyuki_ogata_01.mp4"

in_vid_path = in_fol / in_fn


In [ ]:
img1, img2 = next(pairwise_video_frames(in_vid_path, msec_interval=1000))


In [ ]:

fig, axes = plt.subplots(1, 2)
cv_imshow(img1, ax=axes[0])
cv_imshow(img2, ax=axes[1])


In [ ]:
from climbing_wire.joint_tracker.joint_tracker import JointTracker

pose_img_kwargs = {
    "static_image_mode": True,
    "model_complexity": 2,
}

jt = JointTracker(pose_img_kwargs=pose_img_kwargs)


In [ ]:
jt.process_frame_pair(img1, img2)


## For all frames now

In [ ]:
pose_img_kwargs = {
    "static_image_mode": False,
    "model_complexity": 2,
}

jt = JointTracker(pose_img_kwargs=pose_img_kwargs)


In [ ]:
def_lds = mp_drawing_styles.get_default_pose_landmarks_style()

for idx, (img1, img2) in enumerate(
    # pairwise_video_frames(in_vid_path, msec_interval=1000)
    pairwise_video_frames(in_vid_path, msec_interval=200)
):
    jt.process_frame_pair(img1, img2)
    if jt.landlist is not None:
        ann2 = img2.copy()
        draw_landmarks(ann2, jt.landlist, landmark_drawing_spec=def_lds)
        fig, ax = plt.subplots(1, 1)
        cv_imshow(ann2, ax=ax)
        ax.set_title(f"Frame {idx}")
        plt.show()
    if idx > 10:
        break


In [ ]:
tr_lf = jt.joint_hists['left_foot'].track
tr_lf

In [ ]:
fig, axes = plt.subplots(1, 1)
ax = axes
cv_imshow(img2, ax=ax)
ax.scatter(*tr_lf.T, color="red", s=1)
